#imports

In [3]:
#@title Installations
!pip --quiet install efficientnet_pytorch torchtoolbox

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00


In [4]:
#@title Import package
import pandas as pd
import numpy as np
import os
import ast,json
import requests
from tqdm import tqdm

import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torchtoolbox.transform as transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import WeightedRandomSampler,Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.utils import save_image
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import gc
import os
import cv2
import time
import datetime
import warnings
import random
import matplotlib.pyplot as plt
import seaborn as sns
from efficientnet_pytorch import EfficientNet
%matplotlib inline

In [ ]:
#@title Random Seed
SEED = 4
RANDOM_SEED_TRAIN_TEST = 42

test_size = 0.3
batch_size = 32
np.random.seed(SEED)

In [ ]:
#@title Constants
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#**Drive mount**

## path kaggle: https://www.kaggle.com/datasets/nikdavis/steam-store-games 

In [6]:
from google.colab import drive
drive.mount('/content/drive')
folder = "/content/drive/MyDrive/ML472Project"
data_folder = "/content/drive/MyDrive/ML472Project/Data/"
screenshot_folder = '/content/drive/MyDrive/ML472Project/Data/screenshots/'
screenshot_model_folder = '/content/drive/MyDrive/ML472Project/Models/Screenshots_model/'

Mounted at /content/drive


In [7]:
!ls "/content/drive/MyDrive/ML472Project/Data/"

meta_genre_model.bin	    steam_genres.csv		 steamspy_tag_data.csv
screenshots		    steam_image.csv		 steam_support_info.csv
steam.csv		    steam_media_data.csv
steam_description_data.csv  steam_requirements_data.csv


In [ ]:
#@title Show files
SHOW_BRIEF_OF_FILES = False

if SHOW_BRIEF_OF_FILES:
    files = [f for f in os.listdir(data_folder)]

    for file in files:
        if file.endswith('.csv'):
            df = pd.read_csv(data_folder+file)
            print(file)
            print(df.head())
            print('\n')

# See images and download

---



In [ ]:
#@title Media_df
steam_media_df = pd.read_csv(data_folder+"steam_media_data.csv")
steam_media_df.head()

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN
1,20,https://steamcdn-a.akamaihd.net/steam/apps/20/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN
2,30,https://steamcdn-a.akamaihd.net/steam/apps/30/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/30/...,NaN
3,40,https://steamcdn-a.akamaihd.net/steam/apps/40/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN
4,50,https://steamcdn-a.akamaihd.net/steam/apps/50/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN


In [ ]:
#@title Tag_df
steam_tag_df = pd.read_csv(data_folder+"steamspy_tag_data.csv")
steam_tag_df.head()

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550
1,20,0,71,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,122,0,0,0
3,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,77,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#@title steam metada df
steam_meta_df = pd.read_csv(data_folder +"steam.csv")
steam_meta_df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [19]:
steam_meta_df[steam_meta_df['appid']==1055620]

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
27015,1055620,Minions Battle,2019-04-12,1,RewindApp,RewindApp,windows,0,Single-player;Multi-player;Online Multi-Player,Action;Casual;Indie;Early Access,Early Access;Indie;Casual,0,15,4,0,0,0-20000,0.79


In [ ]:
#@title Choose Rand Image
def choose_random_image(x):
    dir = ast.literal_eval(x)
    len_dir = len(dir)

    image_no = np.random.randint(0,len_dir)
    image_path = dir[image_no]['path_thumbnail']
    
    return image_path

In [ ]:
#@title Select Random image and create new pandas

Select_Images = False 

if Select_Images:
    appid = steam_media_df['steam_appid'].values
    screenshot_url = steam_media_df['screenshots'].apply(choose_random_image).values
    screenshot_path = ["screenshot_"+format(i, 'd')+'.png' for i in appid]

    data = {'appid':appid,'screenshot_url':screenshot_url,'screenshot_path':screenshot_path}

    df = pd.DataFrame(data)

    df.to_csv(data_folder+'steam_image.csv',index=False)

In [ ]:
#@title Download Function
def download(from_url,to_folder,to_path):
    response = requests.get(from_url)
    if response.content:
        None
    else:
        print(to_path)
    open(to_folder+to_path, "wb").write(response.content)

In [ ]:
#@title Download File
Downolad_Screenshots = False

if Downolad_Screenshots:
    for idx in tqdm(df.index):
        download(df.loc[idx,['screenshot_url']].values[0],screenshot_folder,df.loc[idx,['screenshot_path']].values[0])

# Create train and test df

In [ ]:
screenshot_df = pd.read_csv('/content/drive/MyDrive/ML472Project/Data/steam_image.csv')
screenshot_df.head()

,appid,screenshot_url,screenshot_path
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,screenshot_10.png
1,20,https://steamcdn-a.akamaihd.net/steam/apps/20/...,screenshot_20.png
2,30,https://steamcdn-a.akamaihd.net/steam/apps/30/...,screenshot_30.png
3,40,https://steamcdn-a.akamaihd.net/steam/apps/40/...,screenshot_40.png
4,50,https://steamcdn-a.akamaihd.net/steam/apps/50/...,screenshot_50.png


In [ ]:
picked_tags = [
  'indie',
  'action',
  'adventure',
  'casual',
  'singleplayer',
  'strategy',
  'simulation',
  'rpg',
  'puzzle',
  '2d',
  'multiplayer',
  'atmospheric',
  'vr',
  'point_&_click',
  'anime',
  'sports'  
]

steamspy_picked_tags = steam_tag_df[['appid',*picked_tags]]

steamspy_picked_tags.head()

,appid,indie,action,adventure,casual,singleplayer,strategy,simulation,rpg,puzzle,2d,multiplayer,atmospheric,vr,point_&_click,anime,sports
0,10,0,2681,0,0,0,329,0,0,0,0,1659,0,0,0,0,0
1,20,0,208,15,0,0,0,0,0,0,0,172,0,0,0,0,0
2,30,0,99,0,0,16,0,0,0,0,0,115,0,0,0,0,0
3,40,0,85,0,0,0,0,0,0,0,0,58,0,0,0,0,0
4,50,0,211,87,0,148,0,0,0,18,0,0,73,0,0,0,0


In [ ]:
steam_language_df = steam_meta_df[['appid','english']]
steam_language_df.head()

,appid,english
0,10,1
1,20,1
2,30,1
3,40,1
4,50,1


In [ ]:
steam_data_df = pd.merge(pd.merge(steam_language_df,screenshot_df,on='appid'),steamspy_picked_tags,on='appid')
steam_data_df = steam_data_df[steam_data_df["english"] == 1]


print(steam_data_df.shape)
steam_data_df.head()

(26564, 20)


,appid,english,screenshot_url,screenshot_path,indie,action,adventure,casual,singleplayer,strategy,simulation,rpg,puzzle,2d,multiplayer,atmospheric,vr,point_&_click,anime,sports
0,10,1,https://steamcdn-a.akamaihd.net/steam/apps/10/...,screenshot_10.png,0,2681,0,0,0,329,0,0,0,0,1659,0,0,0,0,0
1,20,1,https://steamcdn-a.akamaihd.net/steam/apps/20/...,screenshot_20.png,0,208,15,0,0,0,0,0,0,0,172,0,0,0,0,0
2,30,1,https://steamcdn-a.akamaihd.net/steam/apps/30/...,screenshot_30.png,0,99,0,0,16,0,0,0,0,0,115,0,0,0,0,0
3,40,1,https://steamcdn-a.akamaihd.net/steam/apps/40/...,screenshot_40.png,0,85,0,0,0,0,0,0,0,0,58,0,0,0,0,0
4,50,1,https://steamcdn-a.akamaihd.net/steam/apps/50/...,screenshot_50.png,0,211,87,0,148,0,0,0,18,0,0,73,0,0,0,0


# Dataset and loader

In [ ]:
class Screenshot_Dataset(Dataset):
    def __init__(self, df: pd.DataFrame, root: str, train: bool = True, transforms = None):
        """
        Class initialization
        Args:
            df (pd.DataFrame): DataFrame with data description
            imfolder (str): folder with images
            train (bool): flag of whether a training dataset is being initialized or testing one
            transforms: image transformation method to be applied
            meta_features (list): list of features with meta information, such as sex and age
            
        """
        self.df = df
        self.root = root
        self.transforms = transforms
        self.train = train

    def __getitem__(self, index):
        im_path = os.path.join(self.root, self.df.iloc[index]['screenshot_path'])
        x = cv2.imread(im_path)
        x = cv2.resize(x, (120, 90))

        if self.transforms:
            x = self.transforms(x)
            
        if self.train:
            not_target_col = ['appid','english','screenshot_url','screenshot_path']
            y = self.df.drop(not_target_col,axis=1).iloc[index][:].astype(bool).astype('float64')
            return x, y.values
        else:
            not_target_col = ['appid','english','screenshot_url','screenshot_path']
            y = self.df.drop(not_target_col,axis=1).iloc[index][:].astype(bool).astype('float64')
            return x, y.values
    
    def __len__(self):
        return len(self.df)

In [ ]:
train_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomRotation(
        degrees=45
    ),  # Perhaps a random rotation from -45 to 45 degrees
    transforms.RandomHorizontalFlip(
        p=0.5
    ),  # Flips the image horizontally with probability 0.5
    transforms.RandomVerticalFlip(
        p=0.05
    ),  # Flips image vertically with probability 0.05
    transforms.RandomGrayscale(p=0.2),  # Converts to grayscale with probability 0.2
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3129, 0.3364, 0.3462],std=[0.1862, 0.1876, 0.2007])
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3129, 0.3364, 0.3462],std=[0.1862, 0.1876, 0.2007])
])

In [ ]:
train_df,test_df,_,_ = train_test_split(steam_data_df, [None for _ in range(len(steam_data_df))], 
                                        test_size=test_size, random_state=RANDOM_SEED_TRAIN_TEST, shuffle=True)

In [ ]:
train_dataset = Screenshot_Dataset(df=train_df,root=screenshot_folder,train=True,transforms=train_transform)
test_dataset = Screenshot_Dataset(df=test_df,root=screenshot_folder,train=False,transforms=test_transform)

In [ ]:
train_df.head()

,appid,english,screenshot_url,screenshot_path,indie,action,adventure,casual,singleplayer,strategy,simulation,rpg,puzzle,2d,multiplayer,atmospheric,vr,point_&_click,anime,sports
26011,997720,1,https://steamcdn-a.akamaihd.net/steam/apps/997...,screenshot_997720.png,21,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0
505,24410,1,https://steamcdn-a.akamaihd.net/steam/apps/244...,screenshot_24410.png,0,0,0,0,0,27,0,0,0,0,0,0,0,0,0,0
19017,762610,1,https://steamcdn-a.akamaihd.net/steam/apps/762...,screenshot_762610.png,33,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8171,444930,1,https://steamcdn-a.akamaihd.net/steam/apps/444...,screenshot_444930.png,31,34,0,47,0,0,54,0,0,0,14,0,13,0,0,30
1237,204360,1,https://steamcdn-a.akamaihd.net/steam/apps/204...,screenshot_204360.png,613,920,792,340,0,0,0,498,0,521,730,0,0,0,0,0


In [ ]:
x,y = train_dataset[40]

save_image(x,'img.png')
print(y)

[1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# mean = 0.
# std = 0.
# y_cnt = 0.
# for idx,(images,y) in tqdm(enumerate(train_loader)):
#     batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)
#     y_cnt += y.sum(0)

# mean /= len(train_loader.dataset)
# std /= len(train_loader.dataset)

582it [3:49:45, 23.69s/it]


In [ ]:
mean = [0.3129, 0.3364, 0.3462]
std = [0.1862, 0.1876, 0.2007]
y_cnt = [13933.,  8763.,  7668.,  7719.,  4549.,  4002.,  3941.,  3116.,  2248.,2252.,  2049.,  2017.,  1869.,   799.,  1308.,   985.]

In [ ]:
max_y = max(y_cnt)

13933.0


In [ ]:
weight_of_label = 1/np.array(y_cnt)

In [ ]:
sample_weights = [0] * len(train_dataset)

for idx, (data, label) in enumerate(train_dataset):
    if np.sum(label) == 0:
        sample_weights[idx] = 0
    else :
        class_weight = np.max(np.array(weight_of_label[np.where(label==1)]))
        sample_weights[idx] = class_weight

In [ ]:
sampler = WeightedRandomSampler(
    sample_weights, num_samples=len(sample_weights), replacement=True
)

In [ ]:
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size, sampler=sampler)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

#Architecture

In [ ]:
out_size = len(picked_tags)
l_r = 0.01
load_prev_checkpoint = False
num_epochs = 10

In [ ]:
EFFICIENT_NET_ARCH = EfficientNet.from_pretrained('efficientnet-b1')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b1-f1951068.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b1


In [ ]:
class CNN_Model(nn.Module):
    def __init__(self, arch, out_features: int):
        super(CNN_Model, self).__init__()
        self.arch = arch
        if 'ResNet' in str(arch.__class__):
            self.arch.fc = nn.Linear(in_features=512, out_features=500, bias=True)
        if 'EfficientNet' in str(arch.__class__):
            self.arch._fc = nn.Linear(in_features=1280, out_features=500, bias=True)
        
        self.fc_1 = nn.Linear(500, out_features)
        
    def forward(self, inputs):
        """
        No sigmoid in forward because we are going to use BCEWithLogitsLoss
        Which applies sigmoid for us when calculating a loss
        """
        x = inputs
        cnn_features = self.arch(x)
        fc_1_features = self.fc_1(cnn_features)
        output = torch.sigmoid(fc_1_features)

        return output

In [ ]:
def save_checkpoint(checkpoint,filename='checkpoint.pth.tar'):
  print("=>Saving ......")
  torch.save(checkpoint,filename)
def load_checkpoint(model,optimizer,filename='checkpoint.pth.tar'):
  print("=>Loading ......")
  checkpoint = torch.load(filename)
  model.load_state_dict(checkpoint['model_state'])
  optimizer.load_state_dict(checkpoint['optimizer_state'])

In [ ]:
model = CNN_Model(arch = EFFICIENT_NET_ARCH,out_features = out_size).to(device)
crEnt = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(),lr=l_r)

if load_prev_checkpoint:
  load_checkpoint(model,optimizer)

In [ ]:
for epoch in tqdm(range(num_epochs)):
  losses = []

  if epoch%1 == 0:
    checkpoint = {'model_state':model.state_dict(),'optimizer_state':optimizer.state_dict()}
    save_checkpoint(checkpoint,filename = screenshot_model_folder+f'checkpoint_{epoch}.pth.tar')
  
  for batch_idx,(data,target) in enumerate(train_loader):
    data = data.to(device=device)
    target = target.to(device=device)

    score = model(data)
    loss = crEnt(score,target)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
  mean_loss = sum(losses)/len(losses)
  print(f'epoch no: {epoch} , avgLoss: {mean_loss}')

  0%|          | 0/10 [00:00<?, ?it/s]

=>Saving ......


 10%|█         | 1/10 [03:06<27:58, 186.54s/it]

epoch no: 0 , avgLoss: 11.082491555132766
=>Saving ......


 20%|██        | 2/10 [06:06<24:19, 182.40s/it]

...

epoch no: 7 , avgLoss: 11.017785138012105
=>Saving ......


 90%|█████████ | 9/10 [26:54<02:58, 178.28s/it]

epoch no: 8 , avgLoss: 11.045372875836195
=>Saving ......


100%|██████████| 10/10 [29:52<00:00, 179.29s/it]

epoch no: 9 , avgLoss: 10.991772167691865


In [ ]:
def check_accuracy(loader,model,out_size):
  if loader.dataset.train:
    print('Train Set')
  else:
    print('Test Set')
  num_correct = 0
  num_sample = 0
  model.eval()

  with torch.no_grad():
    for idx,(x,y) in enumerate(loader):
      x = x.to(device=device)
      y = y.to(device=device)

      score = model(x)
      predict = torch.round(score)

      num_correct += (predict==y).sum()/out_size
      num_sample += predict.size(0)

    print(f'Accuracy: {num_correct}/{num_sample} = {num_correct*100.0/num_sample}')

In [ ]:
check_accuracy(train_loader,model,out_size)

Train Set
Accuracy: 13855.75/18594 = 74.51731872558594


In [ ]:
check_accuracy(test_loader,model,out_size)

Test Set
Accuracy: 6226.1875/7970 = 78.12029266357422


In [ ]:
def predict_model(data_loader,model):
    model = model.eval()

    preds_all = []
    targets_all = []

    with torch.no_grad():
        with tqdm(data_loader, unit="batch") as tepoch:
            for x,y in tepoch:
                x = x.to(device=device)
                y = y.to(device=device)
                outputs = model(x)

                preds = torch.round(outputs)
                preds_all.append(preds)
                targets_all.append(y)
    preds_all, targets_all = torch.cat(preds_all), torch.cat(targets_all)
    preds_all, targets_all = preds_all.cpu().detach().numpy(), targets_all.cpu().detach().numpy()
    return preds_all, targets_all

In [ ]:
preds_all, targets_all = predict_model(test_loader,model)

print(classification_report(targets_all,preds_all, target_names = picked_tags))

100%|██████████| 250/250 [1:30:22<00:00, 21.69s/batch]

               precision    recall  f1-score   support

        indie       0.76      1.00      0.86      6022
       action       0.48      1.00      0.65      3823
    adventure       0.42      1.00      0.59      3321
       casual       0.41      1.00      0.58      3238
 singleplayer       0.00      0.00      0.00      1994
     strategy       0.00      0.00      0.00      1666
   simulation       0.00      0.00      0.00      1652
          rpg       0.00      0.00      0.00      1355
       puzzle       0.00      0.00      0.00       954
           2d       0.00      0.00      0.00       946
  multiplayer       0.00      0.00      0.00       866
  atmospheric       0.00      0.00      0.00       843
           vr       0.00      0.00      0.00       847
point_&_click       0.00      0.00      0.00       308
        anime       0.00      0.00      0.00       577
       sports       0.00      0.00      0.00       417

    micro avg       0.51      0.57      0.54     28829
    macr


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
preds_all, targets_all = predict_model(train_loader,model)

print(classification_report(targets_all,preds_all, target_names = picked_tags))

100%|██████████| 582/582 [02:17<00:00,  4.24batch/s]
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

        indie       0.72      1.00      0.84     13465
       action       0.44      1.00      0.61      8139
    adventure       0.47      1.00      0.63      8647
       casual       0.42      1.00      0.59      7811
 singleplayer       0.00      0.00      0.00      6014
     strategy       0.00      0.00      0.00      3825
   simulation       0.00      0.00      0.00      4476
          rpg       0.00      0.00      0.00      3311
       puzzle       0.00      0.00      0.00      3127
           2d       0.00      0.00      0.00      3051
  multiplayer       0.00      0.00      0.00      2743
  atmospheric       0.00      0.00      0.00      2967
           vr       0.00      0.00      0.00      2699
point_&_click       0.00      0.00      0.00      2441
        anime       0.00      0.00      0.00      2380
       sports       0.00      0.00      0.00      2334

    micro avg       0.51      0.49      0.50     77430
    macr

In [ ]:

y_cnt = 0.
for idx,(images,y) in tqdm(enumerate(train_loader)):
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)

    y_cnt += y.sum(0)


NameError: ignored

In [ ]:
print(y_cnt)

tensor([13449.,  8156.,  8548.,  7829.,  5963.,  3962.,  4557.,  3337.,  3161.,
         3083.,  2799.,  2990.,  2684.,  2279.,  2380.,  2428.],
       dtype=torch.float64)
